In [3]:
import numpy as np
import funcs
import os
os.chdir('D://scratch_backup//alpine//beri9208//mnc-pt-pyrid//calcs//surfs//4n//No_bias')

In [2]:
# Feel free to use the savedir option, but it appears that the os.chdir function trumps this option
# and will always end up saving to whatever directory you saved to.
# If your JDFTx calculation was not spin polarized (only have an n file, not n_up and n_dn) just give
# your 'n' file for both the n_up and n_dn fields and change noncollin to True
funcs.chgcar_out('n_up', 'n_dn', 'out', noncollin=False, savedir='C://Users//User//Desktop//', chgcar_name='CHGCAR_normed2')
# the 'pc' boolean just changes how many slashes to put between the directory path and the chgcar file name

In [ ]:
def chgcar_out(n_up, n_dn, out_path, chgcar_name='CHGCAR', savedir=None, pc=True, noncollin=False):
    # Get required variables
    # dtot = np.abs(np.fromfile(n_up))
    # dtot += np.abs(np.fromfile(n_dn))
    dtot = np.fromfile(n_up)
    dtot += np.fromfile(n_dn)
    if noncollin:
        dtot = dtot/2.
    S = funcs.get_grid_shape(out_path)
    ionPos, ionNames, R = funcs.get_coords_vars(out_path)
    # R *= 0.529177 # Convert to Angstroms
    R_inv = R.T
    dtot *= np.dot(R_inv[0], np.cross(R_inv[1], R_inv[2]))
    ion_names, ion_counts = funcs.count_ions(ionNames)
    # Make sure the shapes match up
    try:
        dtot = np.reshape(dtot, S)
        dtot = dtot.flatten(order='F')
    except ValueError:
        print('Error: Ben here! the get_grid_shape function got a shape from your out file that was incompatible with the given d_tot file. Double check and make sure they\'re both from the same calculation, or slack me for help')
    # Write lattice information
    out_str = 'fake_chgcar \n'
    out_str += '   1.0 \n'
    for i in range(3):
        out_str += ' '
        for j in range(3):
            out_str += '    '
            out_str += funcs.fmt_num_1(R[i][j])
        out_str += ' \n'
    # Write ion information
    for name in ion_names:
        out_str += '   '
        out_str += name
    out_str += ' \n'
    for count in ion_counts:
        out_str += '     '
        out_str += str(count)
    out_str += ' \n'
    out_str += 'Direct \n'
    for pos in ionPos:
        for p in pos:
            out_str += '  '
            out_str += funcs.fmt_num_1(p)
        out_str += ' \n'
    out_str += '\n' # Write empty line (?)
    # Write grid shape
    for length in S:
        out_str += '   '
        out_str += str(length)
    out_str += ' \n'
    ticker = 0
    for dens in dtot:
        out_str += ' '
        out_str += funcs.fmt_num_2(dens)
        out_str += '\t'
        if ticker >= 4:
            out_str += ' \n'
            ticker = 0
        else:
            ticker += 1
    out_str += ' \n'
    # Write chgcar
    savepath = ''
    if savedir is not None:
        savepath += savedir
    if pc:
        savepath += '//'
    else:
        savepath += '/'
    savepath += chgcar_name
    with open(chgcar_name, 'w') as chgcarfile:
        chgcarfile.write(out_str)

In [4]:
7 % 5

2

In [ ]:
for i in range(20):